# Project: Building a Custom ChatGPT App with LangChain from Scratch

In [1]:
%%capture 
!pip install -r requirements.txt -q

In [2]:
import os
import openai
import getpass

In [3]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OPENAI_API_KEY: ")

Enter your OPENAI_API_KEY:  ········


In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [11]:
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=1)
prompt = ChatPromptTemplate(
    input_variables=["content"],
    messages=[
        SystemMessage(content="You are chatbot having a conversation with a human."),
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)
# chain = LLMChain(
#     llm=llm,
#     prompt=prompt,
#     verbose=False
# )

chain = prompt | llm | StrOutputParser()

while True:
    content = input("Your prompt: ")
    if content in ["quit", "exit", "bye"]:
        print("Goodbye!")
        break

    response = chain.invoke({"content": content})
    print(response)
    print("-" * 100)

Your prompt:  Kampala is ...


Kampala is the capital city of Uganda, located in East Africa. It is known for its vibrant culture, diverse population, and beautiful landscapes, including hills and lakes. The city has a rich history and offers a mix of modern and traditional influences, evident in its architecture, markets, and cuisine. Some notable attractions include the Uganda National Museum, Kasubi Tombs, and the bustling local markets. Kampala is also a center for education, commerce, and government in Uganda. Is there anything specific you'd like to know about Kampala?
----------------------------------------------------------------------------------------------------


Your prompt:  Tell me about it's history.


Could you please clarify what specific topic or subject you're interested in regarding its history? There are many areas we could explore, such as the history of a particular country, event, invention, or cultural phenomenon. Let me know so I can provide the most relevant information!
----------------------------------------------------------------------------------------------------


Your prompt:  exit


Goodbye!


## Adding Chat Memory Using ConversationBufferMemory

In [19]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

In [24]:
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=1)
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

prompt = ChatPromptTemplate(
    input_variables=["content", "chat_history"],
    messages=[
        SystemMessage(content="You are chatbot having a conversation with a human."),
        MessagesPlaceholder(variable_name="chat_history"), # Where the memory will be stored.
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

# chain = prompt | llm | StrOutputParser()
chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    verbose=False
)

while True:
    content = input("Your prompt: ")
    if content in ["quit", "exit", "bye"]:
        print("Goodbye!")
        break

    response = chain.invoke({"content": content})
    print(response["text"])
    print("-" * 100)

Your prompt:  Paris is ...


Paris is the capital city of France, known for its rich history, art, culture, and fashion. It's home to iconic landmarks like the Eiffel Tower, the Louvre Museum, and Notre-Dame Cathedral. The city is famous for its charming streets, cafes, and vibrant atmosphere, making it a popular destination for tourists from around the world. What specifically would you like to know about Paris?
----------------------------------------------------------------------------------------------------


Your prompt:  Tell me about it's history


Paris has a rich and complex history that dates back over 2,000 years. Here are some key points in its historical timeline:

1. **Foundation and Roman Era (3rd century BC - 5th century AD)**: The area now known as Paris was originally settled by a Celtic tribe called the Parisii around the 3rd century BC. The Romans conquered the city in 52 BC, calling it Lutetia. Under Roman rule, the city grew in significance, with the construction of baths, an amphitheater, and a forum.

2. **Middle Ages (5th century - 15th century)**: After the fall of the Roman Empire, Paris became the capital of the Frankish kingdom. The city was officially declared the capital of France in the early 9th century. By the 12th century, it had become a center for education, with the establishment of the University of Paris. During this period, many iconic structures like Notre-Dame Cathedral began to be constructed.

3. **Renaissance (15th century - 17th century)**: The Renaissance brought a flourishing of arts and 

Your prompt:  exit


Goodbye!


### How to migrate to LangGraph memory

- [How to migrate to LangGraph memory](https://python.langchain.com/docs/versions/migrating_memory/)
- [Migrating off ConversationBufferWindowMemory or ConversationTokenBufferMemory](https://python.langchain.com/docs/versions/migrating_memory/conversation_buffer_window_memory/)

## Saving Chat Sessions

In [26]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage
from langchain.memory import ConversationBufferMemory, FileChatMessageHistory
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

In [27]:
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=1)

# 2. Add an additional keyword argument to the ConversationBufferMemory() constructor
history = FileChatMessageHistory('chat_history.json')
memory = ConversationBufferMemory(
    memory_key="chat_history",
    chat_memory=history,
    return_messages=True
)

prompt = ChatPromptTemplate(
    input_variables=["content", "chat_history"],
    messages=[
        SystemMessage(content="You are chatbot having a conversation with a human."),
        MessagesPlaceholder(variable_name="chat_history"), # Where the memory will be stored.
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

# chain = prompt | llm | StrOutputParser()
chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    verbose=False
)

while True:
    content = input("Your prompt: ")
    if content in ["quit", "exit", "bye"]:
        print("Goodbye!")
        break

    response = chain.invoke({"content": content})
    print(response["text"])
    print("-" * 100)

Your prompt:  Earth mass is ...


The mass of Earth is approximately \(5.97 \times 10^{24}\) kilograms. This value is an estimate based on gravitational measurements and observations of Earth's density and structure. If you have any specific questions or need more details, feel free to ask!
----------------------------------------------------------------------------------------------------


Your prompt:  What about the Moon and the Sun?


The mass of the Moon is about \(7.35 \times 10^{22}\) kilograms, which is roughly 1/81 of Earth's mass.

The mass of the Sun is approximately \(1.99 \times 10^{30}\) kilograms, making it about 333,000 times more massive than Earth. The Sun contains about 99.86% of the total mass of the entire Solar System. If you have more questions or need further information, just let me know!
----------------------------------------------------------------------------------------------------


Your prompt:  What are their diameters?


...

The gravitational acceleration at the surface of Earth, the Moon, and the Sun are approximately as follows:

- **Earth**: About \(9.81 \, \text{m/s}^2\)
- **Moon**: About \(1.62 \, \text{m/s}^2\) (approximately 1/6th that of Earth's gravity)
- **Sun**: About \(274 \, \text{m/s}^2\) (which is significantly stronger due to the Sun's massive size)

These values represent the gravitational pull experienced at the surfaces of these celestial bodies. If you have more questions or need further clarification, let me know!
----------------------------------------------------------------------------------------------------


Your prompt:  What is Newton’s law of gravitation? Write the math and physics equation for Newton’s law of gravitation.


Newton's law of gravitation states that every point mass attracts every other point mass in the universe with a force that is directly proportional to the product of their masses and inversely proportional to the square of the distance between them.

The mathematical equation for Newton's law of universal gravitation is:

\[
F = G \frac{{m_1 m_2}}{{r^2}}
\]

Where:
- \( F \) is the gravitational force between the two masses,
- \( G \) is the gravitational constant, approximately \( 6.674 \times 10^{-11} \, \text{N m}^2/\text{kg}^2 \),
- \( m_1 \) and \( m_2 \) are the masses of the two objects,
- \( r \) is the distance between the centers of the two masses.

This equation shows that the gravitational force decreases as the distance between the masses increases and increases as the mass of either object increases. If you need more details or examples, feel free to ask!
----------------------------------------------------------------------------------------------------


Your prompt:  exit


Goodbye!


In [28]:
from IPython.display import Markdown, display

In [31]:
print(history.messages)

[HumanMessage(content='Earth mass is ...'), AIMessage(content="The mass of Earth is approximately \\(5.97 \\times 10^{24}\\) kilograms. This value is an estimate based on gravitational measurements and observations of Earth's density and structure. If you have any specific questions or need more details, feel free to ask!"), HumanMessage(content='What about the Moon and the Sun?'), AIMessage(content="The mass of the Moon is about \\(7.35 \\times 10^{22}\\) kilograms, which is roughly 1/81 of Earth's mass.\n\nThe mass of the Sun is approximately \\(1.99 \\times 10^{30}\\) kilograms, making it about 333,000 times more massive than Earth. The Sun contains about 99.86% of the total mass of the entire Solar System. If you have more questions or need further information, just let me know!"), HumanMessage(content='What are their diameters?'), AIMessage(content='The approximate diameters of Earth, the Moon, and the Sun are as follows:\n\n- **Earth**: About 12,742 kilometers (7,918 miles)\n- **